<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Details-of-our-datasets" data-toc-modified-id="Details-of-our-datasets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Details of our datasets</a></span></li><li><span><a href="#Model-design" data-toc-modified-id="Model-design-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model design</a></span><ul class="toc-item"><li><span><a href="#Embeddings" data-toc-modified-id="Embeddings-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Embeddings</a></span></li><li><span><a href="#Flattening" data-toc-modified-id="Flattening-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Flattening</a></span></li><li><span><a href="#Shared-layers" data-toc-modified-id="Shared-layers-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Shared layers</a></span></li><li><span><a href="#Merge-layers" data-toc-modified-id="Merge-layers-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Merge layers</a></span></li></ul></li><li><span><a href="#Evaluate-the-model-on-the-tournament-test-data" data-toc-modified-id="Evaluate-the-model-on-the-tournament-test-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Evaluate the model on the tournament test data</a></span></li><li><span><a href="#Three-input-models:-Include-home-vs.-away-games" data-toc-modified-id="Three-input-models:-Include-home-vs.-away-games-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Three-input models: Include home vs. away games</a></span></li><li><span><a href="#Improve-model-by-stacking" data-toc-modified-id="Improve-model-by-stacking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Improve model by stacking</a></span></li><li><span><a href="#Two-prediction-in-one-model" data-toc-modified-id="Two-prediction-in-one-model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Two prediction in one model</a></span></li></ul></div>

The main goal of this work is to design a seed ranking for the teams regarding to the win and loos which is written as  'score_diff' in the dataset. To desing such as ranking we look at all the game and win and loose of the teams and define the new parameter as 'team_strength' which will be added to the data set as a seed difference.

# Details of our datasets

Dataset games_season.csv: Regular season
    
Team ID 1

Team ID 2

Home vs Away

Score Difference (Team 1 -Team 2)

Team 1 Score

Team 2 Score


Dataset 2: Tournament games

Same as Dataset 1

Also has difference in Seed

In [47]:
import pandas as pd
games_season = pd.read_csv('datasets/games_season.csv')
print(games_season.shape)
games_season.head()

(312178, 8)


,season,team_1,team_2,home,score_diff,score_1,score_2,won
0,1985,3745,6664,0,17,81,64,1
1,1985,126,7493,1,7,77,70,1
2,1985,288,3593,1,7,63,56,1
3,1985,1846,9881,1,16,70,54,1
4,1985,2675,10298,1,12,86,74,1


In [48]:
games = pd.read_csv('datasets/games_tourney.csv')
print(games_tourney.shape)
games.head()

(4234, 9)


,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won
0,1985,288,73,0,-3,-9,41,50,0
1,1985,5929,73,0,4,6,61,55,1
2,1985,9884,73,0,5,-4,59,63,0
3,1985,73,288,0,3,9,50,41,1
4,1985,3920,410,0,1,-9,54,63,0


In [3]:
import sys
sys.executable


'C:\\Users\\Hamid\\Anaconda3\\python.exe'

In [4]:
# Import Input from keras.layers
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import pydot

# Create an input layer of shape 1
input_tensor = Input(shape=(1,))

In [10]:
input_tensor = Input(shape=(1,))
output_layer = Dense(1, name='Predicted-Score-Diff')
output_tensor = output_layer(input_tensor)
model = Model(input_tensor, output_tensor)
model.summary()
model.compile(optimizer= 'adam' , loss= 'mae')

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
Predicted-Score-Diff (Dense) (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


# Model design

## Embeddings

Embedding layers increase the dimensionality of the data.
The input CSV file  has two dimensionas of raw and columns, but embedding layers add a thrid dimension. 
Maybe here it is not really relevant but when we work with text and images it is really useful.

1. Input layer: integers
2. Emdedding Layer
3. Output layer: floats

Note: Increased dimensionality: output layer attens back to 2D

For example here we can change the team ID to the decimal rating. we have aound 11000 teams each team  coded as an integer starting with 1 and ending with 100887. We will use this team IDs as inputs to a model that learns the strength of each team.

To use the embedding layer, we need to connect it to the tensor produced by the input layer
## Flattening
They are useful for flattening layer from 3D to 2D. Here we are dealing with numbers so we should bring back the data to 2D by flattening.
The flatten layer is the output layer for the embedding process it means that each time we have the emdedding we should flatten that.


we can save all the process and a model and use it many times.


## Shared layers
Here we have 2 inputs and for both of them we use the the same embedding layer we defined previously. We call the same operation on different inputs as a shared layer


## Merge layers
After we use one model for inputs. we need to mergen this 2 inputs as a single layer so we can predict a signle output

In [23]:
from tensorflow.keras.layers import Embedding
input_tensor = Input(shape=(1,))   #we will call each team as a single integer
n_teams = 10887  #numebr of unique teams in the list
embed_layer = Embedding(input_dim=n_teams, input_length=1, output_dim=1, name='Team-Strength-Lookup') #we name our layer so we can easily find it in the summary
embed_tensor = embed_layer(input_tensor)

In [45]:
# Imports
# In this exercise, you will build a "team strength" layer that represents
# each team by a single number. We will use this number for both teams in the model.
# Imports
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.models import Model
from numpy import unique


# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

We use this model for 2 layers and then we merge them for a single output prediction.

In [87]:
# Imports
from tensorflow.keras.layers import Input, Subtract, Concatenate
from tensorflow.keras.models import Model

# Input layer for team 1
team_in_1 = Input(shape=(1,), name='Team-1-In')

# Separate input layer for team 2
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)


# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')



Now that we've defined a complete team strength model, you can fit it to the basketball data! Since your model has two inputs now, you need to pass the input data as a list.

In [88]:
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']

# Get the team_2 column from the regular season data
input_2 = games_season['team_2']

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1, input_2],
          games_season['score_diff'],
          epochs=100,
          batch_size=2048,
          validation_split=0.10,
          verbose=False)


# Evaluate the model on the tournament test data
The model is fitted to the regular season data (model)  and know we evaluate it with the tournament dataset (games_tourney).

In [89]:
# Get team_1 from the tournament data
input_1 = games_tourney['team_1']

# Get team_2 from the tournament data
input_2 = games_tourney['team_2']

# Evaluate the model using these inputs
print(model.evaluate([input_1, input_2], games_tourney['score_diff'], verbose=False))

10.522969784238848


# Three-input models: Include home vs. away games 

Now We will make an improvement to the model We used so far for regular season games. We want to include the effect of home-away playing , so we will add a new input to our model to capture this effect.

This model will have three inputs: team_id_1, team_id_2, and home. The team IDs will be integers that you look up in your team strength model from the previous chapter, and home will be a binary variable, 1 if team_1 is playing at home, 0 if they are not.


In [92]:
# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [93]:
# Fit the model to the games_season dataset
model.fit([games_season['team_1'], games_season['team_2'], games_season['home']],
          games_season['score_diff'],
          epochs=100,
          verbose=False,
          validation_split=.10,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
print(model.evaluate([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']],
               games_tourney['score_diff'], verbose=False))

9.012384023652867


# Improve model by stacking
It is the using of prediction from one model as an input to another model. 

Process:

1. Train our model by a larger data set of 'regular season'

2. Make a prediction with a smaller dataset of 'tournament data'.

3. Add this prediction to a column of tournament data

4. In the new table with new column we have just he numerical columns, so we choose 3 columns and fit is easily.

In [94]:
# Predict the new tournument data set with the model fited with biger dataset
games_tourney['pred'] = model.predict([games_tourney['team_1'],
                                             games_tourney['team_2'],
                                             games_tourney['home']])

# Add this prediction to a column of tournament data
games_tourney.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won,pred
0,1985,288,73,0,-3,-9,41,50,0,-7.024815
1,1985,5929,73,0,4,6,61,55,1,-0.536807
2,1985,9884,73,0,5,-4,59,63,0,1.298321
3,1985,73,288,0,3,9,50,41,1,2.709431
4,1985,3920,410,0,1,-9,54,63,0,3.460734


In [95]:
# Create an input layer with 2 columns
input_tensor = Input((3,))

# Pass it to a Dense layer with 1 unit
output_tensor = Dense(1)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mae')



In [96]:
from sklearn.model_selection import train_test_split
X = games_tourney[['home','seed_diff','pred']]
y = games_tourney['score_diff']
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

model.fit(train_X,train_y, epochs=100, validation_split=.10,verbose=False)
model.evaluate(test_X, test_y)

1398/1398 [==============================] - 0s 33us/sample - loss: 8.6914


8.691378916111457

# Two prediction in one model

The model that makes two predictions: the scores of both teams in a given game. The inputs will be the seed difference of the two teams, as well as the predicted score difference from the model we built.

In [107]:
# Define the input
input_tensor = Input(shape=(2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(loss='mean_absolute_error', optimizer='adam')

In [108]:
X = games_tourney[['seed_diff','pred']]
y = games_tourney[['score_1','score_2']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)


model.fit(train_X,train_y, epochs=500, verbose=False)

In [109]:
# Print the model's weights
print(model.get_weights())

[array([[ 4.4743996,  3.8619003],
       [-3.3996217, -3.8362129]], dtype=float32), array([43.667473, 43.64047 ], dtype=float32)]
